In [ ]:
#!pip install import-ipynb

In [ ]:
#!pip install ImageHash

In [2]:
#!pip install pyautogui


In [78]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import imagehash
from bs4 import BeautifulSoup
import random
import time
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import random

In [27]:
def ScrollAndLoad(driver):
    # Function to scroll down the page
    def scroll_to_load_all_elements():
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Scroll down to the bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Wait for new content to load
            time.sleep(2)  # Adjust the sleep time as needed
            
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # No more new content
            last_height = new_height
    
    # Call the function to scroll down
    scroll_to_load_all_elements()
    
    # Now you can find the <ul> element
    #ul_element = driver.find_element(By.CSS_SELECTOR, "ul.display-flex.list-style-none.flex-wrap")

    driver.execute_script("window.scrollTo(0, 0);")

In [9]:
#ScrollAndLoad()
def linkedInProfiles(driver):
    # Find the parent <ul> element by its class name
    ul_element = driver.find_element(By.CSS_SELECTOR, "ul.display-flex.list-style-none.flex-wrap")

    # Find all <li> elements within the <ul>
    li_elements = ul_element.find_elements(By.CSS_SELECTOR, "li.grid.grid__col--lg-8.block.org-people-profile-card__profile-card-spacing")

    """Non_existent_linkedIn_Profiles = []
    Non_existent_linkedIn_and_no_Image = []
    LinkedIn_Url = []
    
    for i, li in enumerate(li_elements, start=1):  # Start counting from 1 for better readability
        # Check for the <a> element with the specified class name
        try:
            link_element = li.find_element(By.CSS_SELECTOR, "a.app-aware-link.link-without-visited-state")
            link_url = link_element.get_attribute("href")  # Get the link URL
            LinkedIn_Url.append(link_url)

        except Exception:
            # If the <a> element does not exist, note the position
            Non_existent_linkedIn_Profiles.append(i)
            try:
                # Check for the image element
                missing_image = li.find_element(By.CSS_SELECTOR, "img.evi-image.lazy-image.ghost-person.ember-view")
                Non_existent_linkedIn_and_no_Image.append(i)
            except Exception:
                # No action needed if the image is not found, since it's already captured in the list.
                pass

    #Determeing the index of the list of pictures, where the non-available pictures are not included
    for num in Non_existent_linkedIn_and_no_Image:
        if num in Non_existent_linkedIn_Profiles:
            Non_existent_linkedIn_Profiles.remove(num)  # Remove the number from the first list
            # Decrement numbers larger than the removed number
            Non_existent_linkedIn_Profiles = [x - 1 if x > num else x for x in Non_existent_linkedIn_Profiles]
    """
    Non_existent_linkedIn_Profiles = []
    Non_existent_Image = []
    LinkedIn_Url = []
    memory = []
    #print(enumerate(li_elements))
    for i, li in enumerate(li_elements, start=1):  # Start counting from 1 for better readability
        # Check for the <a> element with the specified class name
        try:
            link_element = li.find_element(By.CSS_SELECTOR, "a.app-aware-link.link-without-visited-state")
            link_url = link_element.get_attribute("href")  # Get the link URL
            LinkedIn_Url.append(link_url)

            try:
                # Check for the image element
                missing_image = li.find_element(By.CSS_SELECTOR, "img.evi-image.lazy-image.ghost-person.ember-view")
                Non_existent_Image.append(i)
            except Exception:
                # No action needed if the image is not found, since it's already captured in the list.
                pass
        except Exception:
            try:
                # Check for the image element
                missing_image = li.find_element(By.CSS_SELECTOR, "img.evi-image.lazy-image.ghost-person.ember-view")
                Non_existent_Image.append(i)
            except Exception:
                # No action needed if the image is not found, since it's already captured in the list.
                pass
            # If the <a> element does not exist, note the position
            Non_existent_linkedIn_Profiles.append(i)
            
    #print("Non_existent_linkedIn_Profiles", Non_existent_linkedIn_Profiles)
    #print("Non_existent_Image", Non_existent_Image)
    #Determeing the index of the list of pictures, where the non-available pictures are not included
    for num in Non_existent_Image:
        if num in Non_existent_linkedIn_Profiles:
            #memory.append(num)
            Non_existent_linkedIn_Profiles.remove(num)  # Remove the number from the first list
            # Decrement numbers larger than the removed number
    #print("Non_existent_linkedIn_Profiles", Non_existent_linkedIn_Profiles)        
    for num in reversed(Non_existent_Image):
        Non_existent_linkedIn_Profiles = [x - 1 if x > num else x for x in Non_existent_linkedIn_Profiles]
    #print("Non_existent_linkedIn_Profiles", Non_existent_linkedIn_Profiles)
    # Print the results without any additional output
    return Non_existent_linkedIn_Profiles, LinkedIn_Url, len(LinkedIn_Url)
# Example usage (assuming 'driver' is already defined and points to the correct page)
#Non_existent_linkedIn_Profiles, LinkedIn_Url, count_existent_profile = linkedInProfiles(driver)
#print(Non_existent_linkedIn_Profiles, LinkedIn_Url)

In [5]:
def load_pictures(Non_existent_linkedIn_Profiles, driver):
    ScrollAndLoad(driver)
    page_source = driver.page_source

    # Use BeautifulSoup to extract image URLs from the HTML source
    from bs4 import BeautifulSoup
    ImageUrl = []
    soup = BeautifulSoup(page_source, 'html.parser')
    images = soup.find_all('img', class_='evi-image lazy-image ember-view')
    for img in images:
        src = img.get('src')
        if "data:image/gif;base64," not in src:
            ImageUrl.append(src)

    # Extract Image URL for missing LinkedIn profiles
    ImageUrl_no_linkedIn = []

    # Iterate through the positions of non-existent LinkedIn profiles
    for img in Non_existent_linkedIn_Profiles:
        # Check if the index is within the bounds of the ImageUrl list
        if img < len(ImageUrl):
            ImageUrl_no_linkedIn.append(ImageUrl[img])
        else:
            print(f"Index {img} is out of range for ImageUrl list and will be skipped.")
    
    return ImageUrl_no_linkedIn

    """ScrollAndLoad(driver)
    page_source = driver.page_source
    # Use BeautifulSoup or regex to extract image URLs from the HTML source
    from bs4 import BeautifulSoup
    ImageUrl = []
    soup = BeautifulSoup(page_source, 'html.parser')
    images = soup.find_all('img', class_='evi-image lazy-image ember-view')
    for img in images:
        src = img.get('src')
        if "data:image/gif;base64," not in src:
            ImageUrl.append(src)
    
    #ImageUrl = ImageUrl[1:]
    #Extract Image URL for missing LinkedIn profiles
    ImageUrl_no_linkedIn = []
    # Iterate through the positions of non-existent LinkedIn profiles
    for img in Non_existent_linkedIn_Profiles:
        # Check if the current position is not in the list of non-existent images
        ImageUrl_no_linkedIn.append(ImageUrl[img])
    #print(ImageUrl_no_linkedIn)
    return ImageUrl_no_linkedIn
#ImageUrl_no_linkedIn = load_pictures() 
#print(ImageUrl_no_linkedIn)
    """

In [33]:
#compare if two images are the same for testing, if the LinkedIn profile is the right one
def compare_images(url1, url2):
    # Fetch the images from the URLs
    response1 = requests.get(url1)
    response2 = requests.get(url2)

    # Open the images
    img1 = Image.open(BytesIO(response1.content))
    img2 = Image.open(BytesIO(response2.content))

    # Optionally resize images to a common size for comparison
    img1 = img1.resize((50, 50))
    img2 = img2.resize((50, 50))

    # Generate perceptual hashes
    hash1 = imagehash.average_hash(img1)
    hash2 = imagehash.average_hash(img2)

    # Compare the hashes
    if hash1 == hash2:
        return True
    else:
        return False

In [96]:
def reversed_image_search(ImageUrl_no_linkedIn, driver):
    # List of image URLs to search
    image_urls = ImageUrl_no_linkedIn
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")  # Open in incognito mode
    options.add_argument("--start-maximized")  # Start the browser maximized
    
    # Create a new instance of the Chrome driver
    driver2 = webdriver.Chrome(options=options)
    
    count_google_lens = 0
    count_google_lens_related_search = 0
    count_not_found = 0
    
    # Open Google Images
    driver2.get("https://images.google.com/")
    time.sleep(3)
    # Wait for the page to load and the reject button to be present, then click it
    try:
        WebDriverWait(driver2, 10).until(EC.presence_of_element_located((By.ID, "W0wltc"))).click()
    except Exception as e:
        print("No reject button found or already clicked.")
    not_found = []#put back at the end of the code again
    personal_profiles = []
    for image_url in image_urls:    
        # Open Google Images
        driver2.get("https://images.google.com/")
    
        # Wait for the camera icon to be clickable
        svg_element = WebDriverWait(driver2, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "svg.Gdd5U"))
        )
    
        # Click the SVG element
        svg_element.click()
    
        # Wait for the "Paste Image URL" input to be visible
        paste_url_input = WebDriverWait(driver2, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "input.cB9M7"))
        )    
        paste_url_input.send_keys(image_url)  # Paste the image URL
        time.sleep(1)  # Wait a moment for any processing
        paste_url_input.send_keys(Keys.ENTER)
        time.sleep(3)
    
        #Test for Related search button, incase exact matches don't exist or don't lead anywhere
        related_search = 0
        try:
            # Wait for the "See exact matches" to be clickable
            svg_element = WebDriverWait(driver2, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div.sZ3jUb"))
            )
            related_search = 1
        except:
            pass
            
        # Open search results
        try:
            # Wait for the "See exact matches" to be clickable
            svg_element = WebDriverWait(driver2, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div.WF9wo"))
            )
            
            # Click the SVG element
            svg_element.click()
        except:
            # Wait for the "See exact matches" to be clickable
            svg_element = WebDriverWait(driver2, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div.ICt2Q"))
            )
            
            # Click the SVG element
            svg_element.click()
    
        time.sleep(1)
    
        def LinkedIn_urls():
            # Get the page source after the search
            page_source = driver2.page_source
        
            # Create a BeautifulSoup object with the page source
            soup = BeautifulSoup(page_source, 'html.parser')
            linkedin_links = []
            # Find all <a> tags and filter for LinkedIn links
            for link in soup.find_all('a', href=True):
                if 'linkedin.com' in link['href']:
                    linkedin_links.append(link['href'])
        
            # Remove duplicates by converting the list to a set, then back to a list
            linkedin_links = list(set(linkedin_links))
            return linkedin_links
    
        linkedin_links = LinkedIn_urls()
        def is_personal_linkedin_profile(link):
            """Check if the given LinkedIn link is a personal profile link."""
            return ("linkedin.com/in/" in link)
    
        # Filter personal profile links
        List = [link for link in linkedin_links if is_personal_linkedin_profile(link)]
        #Identify if LinkedIn is the right LinkedIn of the respective person
        """
        if List != []:
            List2 = []
            for link in List:
                time.sleep(random.uniform(2, 5))
                driver.get(link)
                # Wait for the image element to be visible on the page
                try:
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, 'pv-top-card-profile-picture__image--show'))
                    )
                except Exception as e:
                    print(f"Error loading page for {link}: {e}")
                    continue  # Skip to the next link if there's an error
                page_source = driver.page_source
                # Use BeautifulSoup or regex to extract image URLs from the HTML source
                soup = BeautifulSoup(page_source, 'html.parser')
                # Find the image element (you can use class, id, or other attributes)
                image_element = soup.find('img', class_='pv-top-card-profile-picture__image--show')
                
                # Extract the 'src' attribute
                if image_element:
                    image_src = image_element.get('src')
                    
            
                are_same = compare_images(image_src, image_url)
                if are_same:
                    List2.append(link)
            List = List2
        """
        #Counter for LinkedIn profiles found via Google Lens Web scraping
        if len(List)>0:
            count_google_lens = count_google_lens + 1
        List = List[:3]        
        # If the list is empty and previously tested Related search exists try Related search
        if List == [] and related_search == 1:
            link = ""
            try:
                """# Wait for the "See exact matches" to be clickable
                svg_element = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "div.sZ3jUb"))
                )
                svg_element.click()"""
                page_source = driver2.page_source
                # Create a BeautifulSoup object
                soup = BeautifulSoup(page_source, 'html.parser')
                
                # Use a CSS selector to find the link
                link = soup.select_one('a[href^="https://www.google.com/search?q="]')
                
            except:
                try:
                    driver2.back()
                    """# Wait for the "See exact matches" to be clickable
                    svg_element = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "div.sZ3jUb"))
                    )
                    svg_element.click()"""
                    page_source = driver2.page_source
                    # Create a BeautifulSoup object
                    soup = BeautifulSoup(page_source, 'html.parser')
                    
                    # Use a CSS selector to find the link
                    link = soup.select_one('a[href^="https://www.google.com/search?q="]')
                    
                except:
                    pass
            # Extract the href attribute if link successfully found
            if link != "":
                try:
                    extracted_link = link['href']
                    driver2.get(extracted_link)
                    linkedin_links = LinkedIn_urls()
                    # Filter personal profile links
                    List = [link for link in linkedin_links if is_personal_linkedin_profile(link)]
                except:
                    pass
            if List != []:
                count_google_lens_related_search = count_google_lens_related_search +1
    
        if List == []:
            count_not_found = count_not_found + 1
        personal_profiles.extend(List)
        # Get links that are not personal profiles
        non_personal_links = [link for link in linkedin_links if link not in personal_profiles]
        
        """if len(final_linkedin_links)<1:
            for image in not_found:
                list = load_all_images(image)
                images.extend(list)
        """
    # Output the LinkedIn links found
    return personal_profiles, count_google_lens, count_google_lens_related_search, count_not_found
    
#personal_profiles, count_google_lens, count_google_lens_related_search, count_not_found = reversed_image_search(ImageUrl_no_linkedIn)
#print (personal_profiles)

In [38]:
#Check on LinkedIn if it is a personal profile: looking at the link is slightly less reliable but way faster
"""def linkedInProfiles():
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options
    import time
    
    # Find the parent <ul> element by its class name
    ul_element = driver.find_element(By.CSS_SELECTOR, "ul.display-flex.list-style-none.flex-wrap")
    
    # Find all <li> elements within the <ul>
    li_elements = ul_element.find_elements(By.CSS_SELECTOR, "li.grid.grid__col--lg-8.block.org-people-profile-card__profile-card-spacing")
    
    for li in li_elements:
        # Check for the <a> element with the specified class name
        try:
            link_element = li.find_element(By.CSS_SELECTOR, "a.app-aware-link.link-without-visited-state")
            link_url = link_element.get_attribute("href")  # Get the link URL
            print("Opening link:", link_url)
            #driver.get(link_url)  # Open the link
            #time.sleep(5)  # Wait for the new page to load
            #driver.back()  # Go back to the previous page
            time.sleep(2)  # Wait for the page to reload
        except Exception as e:
            # If the <a> element does not exist, copy the image source
            img_element = li.find_element(By.CSS_SELECTOR, "img.evi-image.lazy-image.ghost-default.ember-view.org-people-profile-card__cover-photo.org-people-profile-card__cover-photo--people")
            img_src = img_element.get_attribute("src")  # Get the image source
            print("Image source:", img_src)"""
    


'def linkedInProfiles():\n    from selenium import webdriver\n    from selenium.webdriver.common.by import By\n    from selenium.webdriver.chrome.service import Service\n    from selenium.webdriver.chrome.options import Options\n    import time\n    \n    # Find the parent <ul> element by its class name\n    ul_element = driver.find_element(By.CSS_SELECTOR, "ul.display-flex.list-style-none.flex-wrap")\n    \n    # Find all <li> elements within the <ul>\n    li_elements = ul_element.find_elements(By.CSS_SELECTOR, "li.grid.grid__col--lg-8.block.org-people-profile-card__profile-card-spacing")\n    \n    for li in li_elements:\n        # Check for the <a> element with the specified class name\n        try:\n            link_element = li.find_element(By.CSS_SELECTOR, "a.app-aware-link.link-without-visited-state")\n            link_url = link_element.get_attribute("href")  # Get the link URL\n            print("Opening link:", link_url)\n            #driver.get(link_url)  # Open the link\n  

In [81]:
"""# Put data in dataframe
def Connect(company_name, counts_df, linkedin_links_df, driver):
    #Call Connector Functions and put the results in a DataFrame
    # Execute Connector methods
    ScrollAndLoad(driver)
    
    # Get LinkedIn profiles and counts
    Non_existent_linkedIn_Profiles, LinkedIn_Url, count_existent_profile = linkedInProfiles(driver)
    #print(Non_existent_linkedIn_Profiles, LinkedIn_Url)

    # Load pictures for non-existent profiles
    ImageUrl_no_linkedIn = load_pictures(Non_existent_linkedIn_Profiles, driver)
    
    # Perform reverse image search
    personal_profiles, count_google_lens, count_google_lens_related_search, count_not_found = reversed_image_search(ImageUrl_no_linkedIn, driver)
    #print(personal_profiles)
    
    # Prepare the total counts
    count_total = count_google_lens + count_google_lens_related_search + count_not_found + count_existent_profile
    count_linkedin = count_google_lens + count_google_lens_related_search + count_existent_profile
    
    # Prepare data for the LinkedIn links DataFrame
    for link in LinkedIn_Url:  # Links from LinkedIn_Url
        linkedin_links_df = pd.concat([linkedin_links_df, 
                                       pd.DataFrame({"Company": [company_name], 
                                                      "LinkedIn Link": [link], 
                                                      "Source": [0]})], ignore_index=True)
    
    for link in personal_profiles:  # Links from personal_profiles
        linkedin_links_df = pd.concat([linkedin_links_df, 
                                       pd.DataFrame({"Company": [company_name], 
                                                      "LinkedIn Link": [link], 
                                                      "Source": [1]})], ignore_index=True)
    
    # Prepare data for the counts DataFrame
    temp_counts_df = pd.DataFrame({
        "Company": [company_name],
        "Total Count": [count_total],
        "Google Lens Count": [count_google_lens],
        "Google Lens Related Search Count": [count_google_lens_related_search],
        "Not Found Count": [count_not_found],
        "Existent Profile Count": [count_existent_profile]
    })
    
    # Concatenate the temporary counts DataFrame with the main counts DataFrame
    counts_df = pd.concat([counts_df, temp_counts_df], ignore_index=True)

    return counts_df, linkedin_links_df
    # After the loop, you can print or return the DataFrames
    #print(linkedin_links_df)
    #print(counts_df
    """

In [14]:
import pandas as pd

def Connect(internal_id, company_name, counts_df, linkedin_links_df, driver):
    try:
        # Execute Connector methods
        ScrollAndLoad(driver)
    
        # Get LinkedIn profiles and counts
        Non_existent_linkedIn_Profiles, LinkedIn_Url, count_existent_profile = linkedInProfiles(driver)
    
        # Load pictures for non-existent profiles
        ImageUrl_no_linkedIn = load_pictures(Non_existent_linkedIn_Profiles, driver)
    
        # Perform reverse image search
        personal_profiles, count_google_lens, count_google_lens_related_search, count_not_found = reversed_image_search(ImageUrl_no_linkedIn, driver)
    
        # Prepare the total counts
        count_total = count_google_lens + count_google_lens_related_search + count_not_found + count_existent_profile
        
        # Collect LinkedIn links data
        linkedin_data = []
        for link in LinkedIn_Url:  # Links from LinkedIn_Url
            linkedin_data.append({"internal_id": internal_id, "Company": company_name, "LinkedIn Link": link, "Source": 0})
        
        for link in personal_profiles:  # Links from personal_profiles
            linkedin_data.append({"internal_id": internal_id, "Company": company_name, "LinkedIn Link": link, "Source": 1})
        
        # Convert the list of dictionaries to a DataFrame
        if linkedin_data:
            temp_linkedin_df = pd.DataFrame(linkedin_data)
            linkedin_links_df = pd.concat([linkedin_links_df, temp_linkedin_df], ignore_index=True)
    
        # Prepare data for the counts DataFrame
        temp_counts_df = pd.DataFrame({
            "internal_id": [internal_id], 
            "Company": [company_name],
            "Total Count": [count_total],
            "Google Lens Count": [count_google_lens],
            "Google Lens Related Search Count": [count_google_lens_related_search],
            "Not Found Count": [count_not_found],
            "Existent Profile Count": [count_existent_profile]
        })
        
        # Concatenate the temporary counts DataFrame with the main counts DataFrame
        counts_df = pd.concat([counts_df, temp_counts_df], ignore_index=True)
    except:
        pass
    return counts_df, linkedin_links_df


In [3]:
# Filter for Founder
def Founder_Filter(driver):
    try:
        searchinput = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID,"people-search-keywords"))
        )
        searchinput.clear()#making sure that there is not text inside because the sended key will be appended
        searchinput.send_keys("founder - founder's associate")
        searchinput.send_keys(Keys.ENTER)
    except Exception as e:
        print("error")